# Plebicito Colombia 02.10.2016

In [1]:
# This is the principal website of the referendum. 
url_main="http://plebiscito.registraduria.gov.co"

# This is the complete url of the website of the referendum. 
url = "http://plebiscito.registraduria.gov.co/99PL/DPLZZZZZZZZZZZZZZZZZ_L1.htm"

In [2]:
from bs4 import BeautifulSoup
import requests

# This function gets the html of the website. 
def get_soup(s):
    r  = requests.get(s)
    data = r.text
    soup = BeautifulSoup(data, "lxml")
    return soup


soup = get_soup(url)

Here we pick the information of each department as a list.
Each element of the list is of type bs4.element.Tag

In [3]:
departments_soup=soup.find_all("option")[56:90]

We make of departments_soup a pandas DataFrame. 
It contains two columns: 
1. The name of the department. 
2. The path to each department result.  

In [6]:
import pandas as pd

departments = pd.DataFrame(columns=("Department", "path"))

for i in range (0,len(departments_soup)):
    row=pd.Series(dict(zip(["Department", "path"],
                           [departments_soup[i].get_text(),
                            departments_soup[i]["value"]])))
    row.name = i
    departments = departments.append(row)

departments

,Department,path
0,AMAZONAS,../99PL/DPL60ZZZZZZZZZZZZZZZ_L1.htm
1,ANTIOQUIA,../99PL/DPL01ZZZZZZZZZZZZZZZ_L1.htm
2,ARAUCA,../99PL/DPL40ZZZZZZZZZZZZZZZ_L1.htm
3,ATLANTICO,../99PL/DPL03ZZZZZZZZZZZZZZZ_L1.htm
4,BOGOTA D.C.,../99PL/DPL16ZZZZZZZZZZZZZZZ_L1.htm
5,BOLIVAR,../99PL/DPL05ZZZZZZZZZZZZZZZ_L1.htm
6,BOYACA,../99PL/DPL07ZZZZZZZZZZZZZZZ_L1.htm
7,CALDAS,../99PL/DPL09ZZZZZZZZZZZZZZZ_L1.htm
8,CAQUETA,../99PL/DPL44ZZZZZZZZZZZZZZZ_L1.htm
9,CASANARE,../99PL/DPL46ZZZZZZZZZZZZZZZ_L1.htm


Looking into the data we see that there are two deparments (Nariño and Norte de Santander) with misspellings. 

In [54]:
departments.loc[22,"Department"] = "NARIÑO"
departments.loc[23,"Department"] = 'NORTE DE SANTANDER'

From each department path we construct the real url and store it in a new column in the deparments DataFrame called "url".

In [64]:
def construct_complete_url(path):
    complete_url = url_main+path.split("..")[1]
    return complete_url

departments["url"]=departments["path"].apply(lambda x: construct_complete_url(x))

In [65]:
departments.head()

,Department,path,url
0,AMAZONAS,../99PL/DPL60ZZZZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
1,ANTIOQUIA,../99PL/DPL01ZZZZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
2,ARAUCA,../99PL/DPL40ZZZZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
3,ATLANTICO,../99PL/DPL03ZZZZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
4,BOGOTA D.C.,../99PL/DPL16ZZZZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...


We create a new column in the deparments DataFrame called "soup". This column stores soup object obtained for each department from the url.

In [66]:
 departments["soup"]=departments["url"].apply(lambda x : get_soup(x))

We make of departments a pandas DataFrame of towns. It contains three columns: 
1. The name of the department. 
2. The name of the town. 
3. The path to each department result. 

In [68]:
towns = pd.DataFrame(columns=("Department", "Town", "path"))

for j in range(0,departments.shape[0]):
    for i in range (0,len(departments["soup"][j].find_all("option")[36:])):
        dept=departments.loc[j,"Department"]
        town_name=departments["soup"][j].find_all("option")[36:][i].get_text()
        town_path=departments["soup"][j].find_all("option")[36:][i]["value"]
        row=pd.Series(dict(zip(["Department", "Town", "path"],[dept,town_name,town_path])))
        towns = towns.append(row, ignore_index=True)

In [71]:
towns.head(20)

,Department,Town,path
0,AMAZONAS,EL ENCANTO,../99PL/DPL60010ZZZZZZZZZZZZ_L1.htm
1,AMAZONAS,LA CHORRERA,../99PL/DPL60013ZZZZZZZZZZZZ_L1.htm
2,AMAZONAS,LA PEDRERA,../99PL/DPL60016ZZZZZZZZZZZZ_L1.htm
3,AMAZONAS,LA VICTORIA,../99PL/DPL60017ZZZZZZZZZZZZ_L1.htm
4,AMAZONAS,LETICIA,../99PL/DPL60001ZZZZZZZZZZZZ_L1.htm
5,AMAZONAS,MIRITI PARANA,../99PL/DPL60019ZZZZZZZZZZZZ_L1.htm
6,AMAZONAS,PUERTO ALEGRIA,../99PL/DPL60030ZZZZZZZZZZZZ_L1.htm
7,AMAZONAS,PUERTO ARICA,../99PL/DPL60040ZZZZZZZZZZZZ_L1.htm
8,AMAZONAS,PUERTO NARIÃO,../99PL/DPL60007ZZZZZZZZZZZZ_L1.htm
9,AMAZONAS,PUERTO SANTANDER,../99PL/DPL60021ZZZZZZZZZZZZ_L1.htm


We create a new column in the towns DataFrame called "url". This coumn stores the real url for each town. 

In [12]:
towns["url"]=towns["path"].apply(lambda x: construct_complete_url(x))

In [13]:
towns.head()

,Department,Town,path,url
0,AMAZONAS,EL ENCANTO,../99PL/DPL60010ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
1,AMAZONAS,LA CHORRERA,../99PL/DPL60013ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
2,AMAZONAS,LA PEDRERA,../99PL/DPL60016ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
3,AMAZONAS,LA VICTORIA,../99PL/DPL60017ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...
4,AMAZONAS,LETICIA,../99PL/DPL60001ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...


In [28]:
def clean_vote(vote_string, null_votes):
    if(null_votes):
        num_v = vote_string.split("Votos nulos")[1]
    else:
        num_v = vote_string.split("(")[1].split( )[0]
    
    num = "".join(num_v.split(".")) 
    return int(num)
    

# We define function which extract the information.

# This function gets the votation information.
def get_vote(u):
    s=get_soup(u)
    
    yes = s.find_all("div", class_="skill-bar-percent")[1].get_text()
    no = s.find_all("div", class_="skill-bar-percent")[3].get_text()
    null = s.find_all("div", class_="cajaInfTercera")[0].find_all("div", class_="contenido")[0].get_text()

    vote = (clean_vote(yes, False), clean_vote(no, False),  clean_vote(null, True))
    
    return vote

We create a new column in the towns DataFrame called "YES". This coumn stores the number of YES in each town.

In [29]:
towns["vote"]=towns["url"].apply(lambda x : get_vote(x))

In [33]:
towns.head()

,Department,Town,path,url,vote,NO,YES,NULL
0,AMAZONAS,EL ENCANTO,../99PL/DPL60010ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...,"(170, 51, 2)",51,170,2
1,AMAZONAS,LA CHORRERA,../99PL/DPL60013ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...,"(300, 62, 3)",62,300,3
2,AMAZONAS,LA PEDRERA,../99PL/DPL60016ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...,"(189, 25, 1)",25,189,1
3,AMAZONAS,LA VICTORIA,../99PL/DPL60017ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...,"(4, 1, 0)",1,4,0
4,AMAZONAS,LETICIA,../99PL/DPL60001ZZZZZZZZZZZZ_L1.htm,http://plebiscito.registraduria.gov.co/99PL/DP...,"(4403, 4463, 84)",4463,4403,84


In [38]:
towns["YES"]=towns["vote"].apply(lambda x : x[0])
towns["NO"]=towns["vote"].apply(lambda x : x[1])
towns["NULL"]=towns["vote"].apply(lambda x : x[2])
towns["TOTAL"]=towns["vote"].apply(lambda x : sum(x))

In [73]:
towns.head()

,Department,Town,path
0,AMAZONAS,EL ENCANTO,../99PL/DPL60010ZZZZZZZZZZZZ_L1.htm
1,AMAZONAS,LA CHORRERA,../99PL/DPL60013ZZZZZZZZZZZZ_L1.htm
2,AMAZONAS,LA PEDRERA,../99PL/DPL60016ZZZZZZZZZZZZ_L1.htm
3,AMAZONAS,LA VICTORIA,../99PL/DPL60017ZZZZZZZZZZZZ_L1.htm
4,AMAZONAS,LETICIA,../99PL/DPL60001ZZZZZZZZZZZZ_L1.htm


We export the towns DataFrame into a .csv file with all the data. 

In [41]:
columns_to_print = ["Department", "Town", "YES", "NO", "NULL", "TOTAL"]

towns[columns_to_print].to_csv("resultados.csv", index=False)